# Tensorflow - TensorBoard

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

C:\Users\Doosik\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Doosik\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Doosik\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Doosik\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: P

### Reinstall protobuf with C++ implementation

To prevent error in protobuf while running tensorboard, protobuf of C++ implementation should be reinstalled.
See https://github.com/tensorflow/tensorboard/issues/1566 for more information.

```bash
set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=cpp
pip install --upgrade --force-reinstall protobuf==3.9.0
```

### Start tensorboard
```bash
tensorboard --logdir=./temp/1/train
```
### Open web browser (with Chrome)
Microsoft Internet Explorer and Microsoft Edge browser does not work correctly!

```bash
"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" http://localhost:6006/
```

## Example: XOR Problem (revised for TensorBoard)

In [13]:
hidden_size = 4

tf.reset_default_graph()

tf.summary.histogram("Total_loss", total_loss)  # Add variable to summary histogram.

# Define input and output placeholders.
X = tf.placeholder(tf.float32, shape=[1, 2], name = 'X')
Y = tf.placeholder(tf.float32, shape=[1, 1], name = 'Y')

# Define weight and bias variables.
W_h = tf.Variable(tf.truncated_normal([2, hidden_size]), name = "W_h")
W_o = tf.Variable(tf.truncated_normal([hidden_size, 1]), name = "W_o")
b_h = tf.Variable(tf.zeros([1, hidden_size]), name = "b_h")
b_o = tf.Variable(tf.zeros([1, 1]), name = "b_o")

# Define new variable to summary.
total_loss = tf.Variable(0.0, name='Total_loss')

In [14]:
N = 4

# Set training data.
INPUT_XOR = [[0,0], [0,1], [1,0], [1,1]]
OUTPUT_XOR = [[0], [1], [1], [0]]

# Define hidden and output layer.
with tf.name_scope("hidden_layer") as scope:
    H = tf.nn.relu(tf.add(tf.matmul(X, W_h), b_h))
with tf.name_scope("output") as scope:
    Y_estimated = tf.sigmoid(tf.add(tf.matmul(H, W_o), b_o))
    
# Define loss function.
with tf.name_scope("loss") as scope:
    loss = tf.reduce_mean(tf.squared_difference(Y_estimated, Y))

# Select optimizer.
with tf.name_scope("train") as scope:
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

# Begin session.
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

tf.summary.histogram("Total_loss", total_loss)  # Add variable to summary histogram.
train_writer = tf.summary.FileWriter( '../temp/1/train', sess.graph)

In [15]:
# Start training.
for epoch in range(10001):
    for i in range(N):
        sess.run(train_step, feed_dict={X: INPUT_XOR[i:i+1], Y: OUTPUT_XOR[i:i+1]})

    if epoch % 1000 == 0:
        loss_value = 0
        for i in range(N):
            loss_value += sess.run(loss, feed_dict={X: INPUT_XOR[i:i+1], Y: OUTPUT_XOR[i:i+1]})

        sess.run(total_loss.assign(loss_value / N))  # Update summary variable.

        merge = tf.summary.merge_all()
        summary = sess.run(merge)
        train_writer.add_summary(summary)

Tensor("Merge/MergeSummary:0", shape=(), dtype=string)
b'\nq\n\x0cTotal_loss_1*a\t\x00\x00\x00 \x00<\xcd?\x11\x00\x00\x00 \x00<\xcd?\x19\x00\x00\x00\x00\x00\x00\xf0?!\x00\x00\x00 \x00<\xcd?) \x00x\xba0\xb5\xaa?2\x18\x05\xc7K\x11?\x1b\xcc?\xedZ\xd3_\xc5\xea\xce?\xff\xff\xff\xff\xff\xff\xef\x7f:\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00'
Tensor("Merge_1/MergeSummary:0", shape=(), dtype=string)
b'\nq\n\x0cTotal_loss_1*a\t\x00\x00\x00\xe0[i\xab?\x11\x00\x00\x00\xe0[i\xab?\x19\x00\x00\x00\x00\x00\x00\xf0?!\x00\x00\x00\xe0[i\xab?) \xc8\xf5\xed%{g?2\x18\xfc\xbe\xe1g\xf0\xe9\xaa?I\x85\xab\xd8\xee\x9a\xad?\xff\xff\xff\xff\xff\xff\xef\x7f:\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00'
Tensor("Merge_2/MergeSummary:0", shape=(), dtype=string)
b'\nq\n\x0cTotal_loss_1*a\t\x00\x00\x00`\xef\\\x91?\x11\x00\x00\x00`\xef\\\x91?\x19\x00\x00\x00\x00\x00\x00\xf0?!\x00\x00\x00`\xef\\\x91?)@F\xcdz\x98\xd

In [16]:
sess.close()

## Example: Linear Regression Problem (revised for TensorBoard)

In [20]:
tf.reset_default_graph()

# Genrating random linear data
# There will be 50 data points ranging from 0 to 50
x = np.linspace(0, 50, 50)
y = np.linspace(20, 50, 50)

# Adding noise to the random linear data
x += np.random.uniform(-4, 4, 50)
y += np.random.uniform(-4, 4, 50)
n = len(x) # Number of data points

In [21]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)
W = tf.Variable(np.random.randn(), name="W")
b = tf.Variable(np.random.randn(), name="b")

learning_rate = 0.01
training_epochs = 500

# Hypothesis
y_pred = X * W + b

# Mean Squared Error Cost Function
cost = tf.reduce_sum(tf.pow(y_pred - Y, 2)) / (2 * n)

# Gradient Descent Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Global Variables Initializer
init = tf.global_variables_initializer()

In [22]:
# Starting the Tensorflow Session
with tf.Session() as sess:

    # Initializing the Variables
    sess.run(init)

    tf.summary.histogram("cost", cost)  # Add variable to summary histogram.
    train_writer = tf.summary.FileWriter( '../temp/1/train', sess.graph)

    # Iterating through all the epochs
    for epoch in range(training_epochs):

        # Feeding each data point into the optimizer using Feed Dictionary
        for (_x, _y) in zip(x, y):
            sess.run(optimizer, feed_dict = {X : _x, Y : _y})

        # Displaying the result after every 50 epochs
        if (epoch + 1) % 50 == 0:
            # Calculating the cost a every epoch
            c = sess.run(cost, feed_dict = {X : x, Y : y})
            print("Epoch", (epoch + 1), ": cost =", c, "W =", sess.run(W), "b =", sess.run(b))

            merge = tf.summary.merge_all()
            summary = sess.run(merge)
            train_writer.add_summary(summary, epoch)

Epoch 50 : cost = 45.899097 W = 1.0389813 b = 2.263684


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float
	 [[node Placeholder (defined at <ipython-input-21-f9963ebe2c85>:1)  = Placeholder[dtype=DT_FLOAT, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'Placeholder', defined at:
  File "C:\Users\Doosik\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Doosik\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Doosik\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Doosik\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Doosik\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Users\Doosik\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 148, in star

## Reference
- https://www.tensorflow.org/guide/summaries_and_tensorboard